In [ ]:
### Q4
import numpy as np
import pandas as pd

In [ ]:
conf_matrix = [
    [800, 30, 110],
    [140, 50, 70 ],
    [100, 20, 90 ]
]
conf_matrix = np.array(conf_matrix)
features = ['monthly', 'quarterly', 'yearly']

In [ ]:
conf_matrix[:,0]

In [ ]:
## (a) low level
def average_class_accuracy(conf_matrix):
    # Apparently (refer Instruction 6) this is just the average of the recalls
    # of each class
    return sum(recall(i,conf_matrix) for i in range(len(features)))/len(features)

def accuracy(conf_matrix):
    # Accuracy = (TP+TN)/(TP+TN+FP+FN), but I don't think that can apply for
    # a multiclass confusion matrix.
    pass

def recall(class_,conf_matrix):
    # Recall = True Positive Rate, something I know how to do of the top of my hat
    # but would have been completely lost in the exam because I didn't remember what recall
    # is (ironic). All exams should be replaced with assignments they are not representetive
    # of your abilities.
    
    # True positives on the diagonal
    TP = conf_matrix[class_,class_]
    # Positives in the row
    P = sum(conf_matrix[class_])
    return TP/P

def precision(class_,conf_matrix):
    # Wasn't asked but seems relevant
    # True positives on the diagonal
    TP = conf_matrix[class_,class_]
    # False positives in the column
    FP = sum(conf_matrix[:,class_]) - TP
    return TP/(TP+FP)

for i,v in enumerate(features):
    print(f"Recall for plan '{v}': {recall(i,conf_matrix):.2%}")
    print(f"Precision for plan '{v}': {precision(i,conf_matrix):.2%}")

print(f"Average class accuracy: {average_class_accuracy(conf_matrix):.2%}")

In [ ]:
## (b) data
before = """A B C plan
1 2 1 monthly
1 1 2 quarterly
0 0 0 monthly
1 0 0 quarterly
0 1 0 quarterly
1 2 1 monthly
1 5 2 quarterly
1 1 4 quarterly
0 2 5 quarterly
0 3 1 monthly
1 2 1 monthly
0 3 1 yearly"""
after = """A B C plan
2 1 0 monthly
1 2 1 quarterly
0 3 0 monthly
0 5 1 monthly
0 0 6 quarterly
0 1 1 yearly
0 2 1 yearly
1 5 2 yearly
2 7 3 monthly
1 8 4 quarterly
0 5 5 monthly"""
before = before.splitlines()

columns = before[0].split()
col_types = [int,int,int,str]

before = [l.split() for l in before[1:]]
before = pd.DataFrame(before, columns=columns)
before = before.astype(dict(zip(columns,col_types)))

after = after.splitlines()[1:]
after = [l.split() for l in after]
after = pd.DataFrame(after, columns=columns)
after = after.astype(dict(zip(columns,col_types)))

display(before.head())
display(after.head())

In [ ]:
## (b) Dafuq is a system stability index?
# Answer: A system stability index is (fraction of class before - fraction of class after) times the log of (fraction of class before/fraction of class after),
# summed over all classes.

# "Compute the system stability index between before and after release" <- Learn to write, I was like "How do I do that before or after when it requires two
# to compare?". I think it means "Compute the system stability index for the change from before to after release.""

def system_stability_index(before, after, summed=True):
    ssi = []
    ssi_plan = []
    for plan in set(before['plan']) & set(after['plan']):
        plan_percent_before = sum(before['plan'] == plan)/len(before)
        plan_percent_after = sum(after['plan'] == plan)/len(after)
        ssi.append((plan_percent_before - plan_percent_after) * np.log(plan_percent_before/plan_percent_after))
        ssi_plan.append(plan)
    if summed:
        return sum(ssi)
    else:
        return zip(ssi_plan, ssi)

print(f"System stability index: {system_stability_index(before, after):.2f}")
for plan, ssi in system_stability_index(before, after, summed=False):
    print(f"System stability index for plan '{plan}': {ssi:.2f}")